# Confidence Score Calculation and Prediciton

In [ ]:
import pandas as pd
results = pd.read_csv('./data/apiResultsOver18.csv')

data = pd.read_csv('./data/data.csv')

perno = pd.read_csv('./data/perno.csv')

addre = pd.read_csv('./data/ad.csv')

In [ ]:
perno.head()

## Name Commonality

In [ ]:
# import numpy as np
# from module.confidenceScore import similarity
# names = np.unique(pernor['FIRST_NAME'])
# uniquePairs = [[a, b, similarity(a,b)] for idx, a in enumerate(names) for b in names[idx + 1:]]

In [ ]:
import numpy as np
def nameCommonality(names):
    uniqueNames,uniqueNamesCount = np.unique(names,return_counts=True)
    uniqueNamesSum = np.sum(uniqueNamesCount)
    uniqueNamesCount = (uniqueNamesCount / uniqueNamesSum) * 100
    nameCommonalityDict = dict(zip(uniqueNames, uniqueNamesCount)) 

    commonality = [nameCommonalityDict[name] for name in names]
    
    return commonality 

In [ ]:
perno['FIRST_NAME_COMMONALITY'] = nameCommonality(list(perno['FIRST_NAME']))
perno['SURNAME_COMMONALITY'] = nameCommonality(list(perno['SURNAME']))

data['firstNameCommonality'] = perno['FIRST_NAME_COMMONALITY']
data['surnameCommonality'] = perno['SURNAME_COMMONALITY']

In [ ]:
datar = data[data['perno'].isin(results['perno'])]
pernor = perno[perno['PERNO'].isin(results['perno'])]
addr = addre[addre['PERNO'].isin(results['perno'])]

## Calc Scores

In [ ]:
from module.confidenceScore import confidenceScoreCalc
import numpy as np

confidenceScores = confidenceScoreCalc(results, pernor, addr)

results['confidenceScore'] = confidenceScores
confidenceScores = results[['perno','confidenceScore']]

pernos = list(datar['perno'])
maxScore=[]
for p in pernos:
    maxScore.append(np.max(confidenceScores[confidenceScores['perno'] == p]['confidenceScore']))
datar['traceability'] = maxScore

## ML model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

df = datar[['valueLog','moveFreq','premsFreq','premsMean','premsMedian','premsMax','premsMin','goneAway','firstNameCommonality','surnameCommonality','traceability']]

df1 = datar[['valueLog','moveFreq','premsFreq','premsMean','premsMedian','premsMax','premsMin','goneAway','firstNameCommonality','surnameCommonality','traceability']]
df1.loc[(df1.traceability > 0), 'traceability'] = 1
df2 = df.where(df["traceability"] > 0).dropna()

X1 = MinMaxScaler().fit_transform(df1[['valueLog','moveFreq','premsFreq','premsMedian','goneAway','firstNameCommonality','surnameCommonality']].to_numpy())
X2 = MinMaxScaler().fit_transform(df2[['valueLog','moveFreq','premsFreq','premsMedian','goneAway','firstNameCommonality','surnameCommonality']].to_numpy())

y1 = df1[['traceability']].to_numpy()
y2 = df2[['traceability']].to_numpy()

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.4, random_state=1)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.4, random_state=1)


kf = KFold(n_splits=3)
#reg = SVR(kernel = 'linear')
#scvLinearScore = SVC(kernel = 'linear').fit(X1, y1).score

#reg = RandomForestRegressor().fit(X2, y2)

## Classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svc_param = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'max_iter': [-1,1,5,10,20,40]}

clf = GridSearchCV(SVC(), svc_param, cv=2)

clf.fit(X1_train, y1_train.ravel())
clf.best_params_

In [ ]:
clf.score(X1_test,y1_test)

In [ ]:
SVC(kernel = 'sigmoid').fit(X1_train,y1_train).score(X1_test,y1_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

tree = DecisionTreeClassifier()

tree_param = {'criterion': ['entropy', 'gini'], 
               'max_depth': [2,4,8,16,32,64,128],
              'min_samples_leaf': [1,2,3,4,5]}

clf = GridSearchCV(DecisionTreeClassifier(random_state=1), tree_param, cv=9)

clf.fit(X1_train, y1_train)
clf.score(X1_test,y1_test)
#treeClsScore = cross_val_score(tree, X1, y1, cv=3).mean()

In [ ]:
clf.predict(X1_test)

In [ ]:
scores = [GridSearchCV(DecisionTreeClassifier(random_state=1), tree_param, cv=i).fit(X1_train, y1_train.ravel()).score(X1_test,y1_test) for i in range(2,15)]
scores

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = GridSearchCV(RandomForestClassifier(random_state = 57), tree_param, cv=2)

clf.fit(X1_train, y1_train.ravel())
clf.predict(X1_test)

In [ ]:
clf.score(X1_test,y1_test)

In [ ]:
clf.best_params_

In [ ]:
# scores = []

# for i in range(2,8):
#     clf = GridSearchCV(RandomForestClassifier(), tree_param, cv=i)

#     clf.fit(X1_train, y1_train.ravel())
#     scores.append(clf.score(X1_test,y1_test))

In [ ]:
scores

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
NBClsScore = cross_val_score(nb, X1, y1.ravel(), cv=3).mean()
NBClsScore

In [ ]:
import pickle
model = RandomForestClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 1,random_state=57).fit(X1_train, y1_train.ravel())
pickle.dump(model,open('./models/RFClsConfidenceScore_25-10-21.pickle', 'wb'))

## Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_param = {'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
              'max_depth': [2,4,8,16,32],
              'min_samples_leaf': [1,2,3,4,5]}

reg = GridSearchCV(DecisionTreeRegressor(), tree_param, cv=5)

reg.fit(X2_train, y2_train.ravel())
reg.score(X2_test,y2_test)

In [ ]:
reg.best_params_

In [ ]:
from sklearn.ensemble import RandomForestRegressor

reg = GridSearchCV(RandomForestRegressor(), tree_param, cv=2)

reg.fit(X2_train, y2_train.ravel())
reg.score(X2_test,y2_test)

In [ ]:
reg.best_params_

In [ ]:
from sklearn import svm

reg = GridSearchCV(svm.SVR(), svc_param, cv=2)

reg.fit(X2_train, y2_train.ravel())
reg.score(X2_test,y2_test)

In [ ]:
reg.best_params_

In [ ]:
from sklearn import svm

svmScore = svm.SVR(kernel='linear',max_iter=-1).fit(X2_train, y2_train.ravel()).score(X2_test, y2_test)
svmScore

In [ ]:
min(svm.SVR(kernel='linear',max_iter=-1).fit(X2_train, y2_train.ravel()).predict(X2_test))

In [ ]:
reg = RandomForestRegressor(criterion = 'friedman_mse', max_depth = 4, min_samples_leaf = 5,random_state=55).fit(X2_train,y2_train)
reg.score(X2_test,y2_test)

In [ ]:
model = svm.SVR(kernel='linear',max_iter=-1).fit(X2_train, y2_train.ravel())
pickle.dump(model,open('./models/SvmRegConfidenceScore_25-10-21.pickle', 'wb'))

## Apply to full data, get distr

In [ ]:
import pickle
import pandas as pd

cls = pickle.load(open('./models/RFClsConfidenceScore_25-10-21.pickle', 'rb'))
reg = pickle.load(open('./models/SvmRegConfidenceScore_25-10-21.pickle', 'rb'))

In [ ]:
X1 = data[['valueLog','moveFreq','premsFreq','premsMedian','goneAway','firstNameCommonality','surnameCommonality']].to_numpy()
y1 = cls.predict( MinMaxScaler().fit_transform(X1))
data['traceability'] = y1

In [ ]:
np.mean(y1)

In [ ]:
data.to_csv('./data/dataTrace.csv')

In [ ]:
df2 = data.where(data["traceability"] > 0).dropna(how='all')
X2 = df2[['valueLog','moveFreq','premsFreq','premsMedian','goneAway','firstNameCommonality','surnameCommonality']].to_numpy()
y2 = reg.predict( MinMaxScaler().fit_transform(X2))

In [ ]:
df2['traceability'] = y2

df1 = data.where(data["traceability"] == 0).dropna(how='all')

df = pd.concat([df1,df2])

df.sort_index()

In [ ]:
df.describe().to_csv('./data/traceabilityDesc.csv')

In [ ]:
from itertools import combinations
import matplotlib.pyplot as plt

fig, axes = plt.subplots(len(df.columns)//3, 3, figsize=(12, 48))

i = 0
for triaxis in axes:
    for axis in triaxis:
        df.hist(column = df.columns[i], bins = 10, ax=axis)
        i = i+1

In [ ]:
import seaborn

seaborn.heatmap(df.corr())

In [ ]:
df.corr().to_csv('./data/covar21.csv')